# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [8]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# 2. Load messages.csv into a dataframe
messages = pd.read_csv('/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_messages.csv')
messages.head()
# Inspect the first few lines of the messages dataframe
print("Messages DataFrame:")
print(messages.head())

# 3. Load categories.csv into a dataframe
categories = pd.read_csv('/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_categories.csv')
categories = pd.concat([categories, categories['categories'].str.split(';', expand=True)], axis=1)

# Inspect the first few lines of the categories dataframe
print("\nCategories DataFrame:")
print(categories.head())

Messages DataFrame:
   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct  

Categories DataFrame:
   id                                         categories
0   2  related-1;request-0;offer-0;aid_related-0;medi...
1   7  related-1;request-0;offer-0;aid_related-1;medi...
2   8  related-1;request-0;offer-0;aid_related-

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [13]:
# merge datasets
df = pd.merge(categories, messages, on='id')
df.head()

,id,categories,message,original,genre
0,2,related-1;request-0;offer-0;aid_related-0;medi...,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,related-1;request-0;offer-0;aid_related-1;medi...,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,related-1;request-0;offer-0;aid_related-0;medi...,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,related-1;request-1;offer-0;aid_related-1;medi...,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,related-1;request-0;offer-0;aid_related-0;medi...,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [8]:
# create a dataframe of the 36 individual category columns
categories = pd.concat([categories, categories['categories'].str.split(';', expand=True)], axis=1)
categories.head()

,id,categories,0,1,2,3,4,5,6,7,...,26,27,28,29,30,31,32,33,34,35
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [9]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x: str(x).split('-')[0])

print(category_colnames)

id                                 2
categories                   related
0                            related
1                            request
2                              offer
3                        aid_related
4                       medical_help
5                   medical_products
6                  search_and_rescue
7                           security
8                           military
9                        child_alone
10                             water
11                              food
12                           shelter
13                          clothing
14                             money
15                    missing_people
16                          refugees
17                             death
18                         other_aid
19            infrastructure_related
20                         transport
21                         buildings
22                       electricity
23                             tools
24                         hospitals
2

In [10]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,2,related,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [11]:
for column in df.columns[4:]:
    # Set each value to be the last character of the string
    df[column] = df[column].astype(str).str[-1]
    
    # Print unique values in the column
    unique_values = df[column].unique()
    print(f"Column: {column}, Unique Values: {unique_values}")
    
    # Convert the string to numeric
    df[column] = pd.to_numeric(df[column], errors='coerce')  # Use errors='coerce' to handle non-numeric values by converting them to NaN

# Display the first few rows of the modified dataframe
df.head()


Column: genre, Unique Values: ['t' 'l' 's']


,id,categories,message,original,genre
0,2,related-1;request-0;offer-0;aid_related-0;medi...,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,NaN
1,7,related-1;request-0;offer-0;aid_related-1;medi...,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,NaN
2,8,related-1;request-0;offer-0;aid_related-0;medi...,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",NaN
3,9,related-1;request-1;offer-0;aid_related-1;medi...,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,NaN
4,12,related-1;request-0;offer-0;aid_related-0;medi...,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,NaN


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [12]:
# Replace 'categories' column in df with new category columns
df = pd.concat([df, categories], axis=1)

# Drop the original 'categories' column from the df dataframe
df = df.drop('categories', axis=1)

# Display the first few lines of the dataframe after dropping the original 'categories' column
print(df.head())


   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original  genre     2  \
0  Un front froid se retrouve sur Cuba ce matin. ...    NaN   2.0   
1                 Cyclone nan fini osinon li pa fini    NaN   7.0   
2  Patnm, di Maryani relem pou li banm nouvel li ...    NaN   8.0   
3  UN reports Leogane 80-90 destroyed. Only Hospi...    NaN   9.0   
4  facade ouest d Haiti et le reste du pays aujou...    NaN  12.0   

                                             related    related    request  \
0  related-1;request-0;offer-0;aid_related-0;medi...  related-1  request-0   
1  related-1;request-0;offer-0;aid_related-1;medi...  related-1  req

In [13]:
# Concatenate the original dataframe with the new 'categories' dataframe
df = pd.concat([df, categories], axis=1)

# Display the first few lines of the modified dataframe
print(df.head())


   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original  genre     2  \
0  Un front froid se retrouve sur Cuba ce matin. ...    NaN   2.0   
1                 Cyclone nan fini osinon li pa fini    NaN   7.0   
2  Patnm, di Maryani relem pou li banm nouvel li ...    NaN   8.0   
3  UN reports Leogane 80-90 destroyed. Only Hospi...    NaN   9.0   
4  facade ouest d Haiti et le reste du pays aujou...    NaN  12.0   

                                             related    related    request  \
0  related-1;request-0;offer-0;aid_related-0;medi...  related-1  request-0   
1  related-1;request-0;offer-0;aid_related-1;medi...  related-1  req

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [14]:
# Check how many duplicates are in the dataset
duplicates_before = df.duplicated().sum()
print("Number of duplicates before removal:", duplicates_before)

Number of duplicates before removal: 0


In [15]:
# Drop the duplicates
df = df.drop_duplicates()
print(df.columns)

df = df.loc[:,~df.columns.duplicated()]
print(df.columns)

# Rimuoviamo le colonne '2' e 'related'
df = df.drop(['2', 'related'], axis=1)

print(df.columns)

Index(['id', 'message', 'original', 'genre', '2', 'related', 'related',
       'request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report',
       '2', 'related', 'related', 'request', 'offer', 'aid_related',
       'medical_help', 'medical_products', 'search_and_rescue', 'security',
       'military', 'child_alone', 'water', 'food', 'shelter', 'clothing',
       'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_cent

In [16]:
# Check how many duplicates are in the dataset after removal
duplicates_after = df.duplicated().sum()
print("Number of duplicates after removal:", duplicates_after)

# Confirm duplicates were removed by displaying the shape of the dataframe
print("Shape of the dataframe after removing duplicates:", df.shape)

Number of duplicates after removal: 60
Shape of the dataframe after removing duplicates: (26386, 39)


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [17]:
# Create an SQLite engine
engine = create_engine('sqlite:///samtable.db')

# Save the clean dataset into the SQLite database
df.to_sql('samtable', engine, index=False)

# Print a message indicating the successful save
print("Clean dataset saved to SQLite database.")



Clean dataset saved to SQLite database.


### 8. Use this notebook to complete `process_data.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `process_data.py` in the classroom on the `Project Workspace IDE` coming later.

In [37]:
import sys
import pandas as pd
from sqlalchemy import create_engine

# database_filepath = sys.argv[1:]
# database_filepath = 'DisasterResponse.db'


def load_data_cat(categories_filepath):
    """Carica i dati delle categorie da un file CSV."""
    categories = pd.read_csv(categories_filepath)
    return categories

def load_data_mes(messages_filepath):
    """Carica i dati dei messaggi da un file CSV."""
    messages = pd.read_csv(messages_filepath)
    return messages

def clean_data(df, categories):
    """Pulisce i dati combinando i dataframe dei messaggi e delle categorie."""
    # Unisci i dataframe dei messaggi e delle categorie
    df = pd.merge(df, categories, on='id')
    
    # Split delle colonne delle categorie
    categories = df['categories'].str.split(';', expand=True)
    
    # Estrai i nomi delle colonne
    row = categories.iloc[0]
    category_colnames = row.apply(lambda x: x.split('-')[0]).values
    
    # Rinomina le colonne delle categorie
    categories.columns = category_colnames
    
    # Converti i valori delle categorie in numeri binari
    for column in categories:
        categories[column] = categories[column].apply(lambda x: int(x.split('-')[1]))
    
    # Sostituisci la colonna 'categories' con il dataframe delle categorie
    df = df.drop('categories', axis=1)
    df = pd.concat([df, categories], axis=1)
    
    # Rimuovi i duplicati
    df.drop_duplicates(inplace=True)
    
    return df

def save_data(df, db_filename):
    engine = create_engine(f'sqlite:///{db_filename}')
    
    # Save the clean dataset into the SQLite database
    df.to_sql('samtable', engine, index=False, if_exists='replace')
    
    # Print a message indicating the successful save
    print("Clean dataset saved to SQLite database.")

def main():
    """Punto di ingresso principale."""
    if len(sys.argv) == 4:
        mes_file_path = '/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_messages.csv'
        cat_file_path = '/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_categories.csv'
        database_filepath = 'DisasterResponse.db'
        
        print('Caricamento dei dati...')
        messages = load_data_mes(mes_file_path)
        categories = load_data_cat(cat_file_path)
        
        print('Pulizia dei dati...')
        df = clean_data(messages, categories)
        
        print('Salvataggio dei dati...')
        save_data(df=df,db_filename=database_filepath)
        
        print('I dati puliti sono stati salvati nel database SQLite!')
    else:
        print('Per favore, fornisci i percorsi dei file dei messaggi, delle categorie e del database SQLite '
              'come argomenti nella riga di comando.')

if __name__ == '__main__':
    main()


Per favore, fornisci i percorsi dei file dei messaggi, delle categorie e del database SQLite come argomenti nella riga di comando.


In [38]:
# TEST

import pandas as pd
from sqlalchemy import create_engine

def load_data_cat(categories_filepath):
    """Carica i dati delle categorie da un file CSV."""
    categories = pd.read_csv(categories_filepath)
    return categories

def load_data_mes(messages_filepath):
    """Carica i dati dei messaggi da un file CSV."""
    messages = pd.read_csv(messages_filepath)
    return messages

def clean_data(df, categories):
    """Pulisce i dati combinando i dataframe dei messaggi e delle categorie."""
    # Unisci i dataframe dei messaggi e delle categorie
    df = pd.merge(df, categories, on='id')
    
    # Split delle colonne delle categorie
    categories = df['categories'].str.split(';', expand=True)
    
    # Estrai i nomi delle colonne
    row = categories.iloc[0]
    category_colnames = row.apply(lambda x: x.split('-')[0]).values
    
    # Rinomina le colonne delle categorie
    categories.columns = category_colnames
    
    # Converti i valori delle categorie in numeri binari
    for column in categories:
        categories[column] = categories[column].apply(lambda x: int(x.split('-')[1]))
    
    # Sostituisci la colonna 'categories' con il dataframe delle categorie
    df = df.drop('categories', axis=1)
    df = pd.concat([df, categories], axis=1)
    
    # Rimuovi i duplicati
    df.drop_duplicates(inplace=True)
    
    return df

def save_data(df, db_filename):
    engine = create_engine(f'sqlite:///{db_filename}')
    
    # Save the clean dataset into the SQLite database
    df.to_sql('samtable', engine, index=False, if_exists='replace')
    
    # Print a message indicating the successful save
    print("Clean dataset saved to SQLite database.")

def main():
    """Punto di ingresso principale."""
    # Imposta i percorsi dei file e il nome del database predefiniti
    mes_file_path = '/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_messages.csv'
    cat_file_path = '/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_categories.csv'
    database_filepath = 'DisasterResponse.db'
    
    print('Caricamento dei dati...')
    messages = load_data_mes(mes_file_path)
    categories = load_data_cat(cat_file_path)
    
    print('Pulizia dei dati...')
    df = clean_data(messages, categories)
    
    print('Salvataggio dei dati...')
    save_data(df=df, db_filename=database_filepath)
    
    print('I dati puliti sono stati salvati nel database SQLite!')

if __name__ == '__main__':
    main()


Caricamento dei dati...
Pulizia dei dati...
Salvataggio dei dati...
Clean dataset saved to SQLite database.
I dati puliti sono stati salvati nel database SQLite!


In [ ]:
def main():
    """Main entry point."""
    print('inzio ciclo:')
    if len(sys.argv) == 4: #questo controlla che ci siano 4 argomenti quando viene lanciato dalla riga di comando
        mes_file_path = '/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_messages.csv'
        cat_file_path = '/Users/samuelmilazzo/Desktop/samuel 2023/learning/project2/disaster_response_pipeline_project/data/disaster_categories.csv'
        database_filepath = 'DisasterResponse.db'

        print('Loading data...')
        messages = load_data_mes(mes_file_path)
        categories = load_data_cat(cat_file_path)

        print('Cleaning data...')
        df = clean_data(messages, categories)

        print('Saving data...')
        save_data(df=df, db_filename=database_filepath)

        print('Cleaned data saved to the SQLite database!')
    else:
        print('Please provide the filepaths of the messages, categories, and SQLite database '
              'as arguments in the command line.')


if __name__ == '__main__':
    main()
 